In [7]:
import os
import pandas as pd
import numpy as np

# Define required features and corresponding sheet names
required_features = {
    "Net Income Continuous Operations": "Income Statement",
    "Total Revenue": "Income Statement",
    "Stockholders Equity": "Balance Sheet",
    "Total Assets": "Balance Sheet",
    "Current Assets": "Balance Sheet",
    "Current Liabilities": "Balance Sheet",
    "Inventory": "Balance Sheet",  # May be missing, default to 0
    "Total Debt": "Balance Sheet",
    "Interest Expense": "Income Statement",
    "EBIT": "Income Statement"
}

# Directory containing Excel files
folder_path = "Financial docs"  # Update this

# Initialize list to store processed data
final_data = []

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)
        
        try:
            # Load both sheets
            xls = pd.ExcelFile(file_path)
            data = {}

            # Extract required features
            for feature, sheet in required_features.items():
                try:
                    df = xls.parse(sheet)
                    if feature in df.iloc[:, 0].values:
                        value = df[df.iloc[:, 0] == feature].iloc[:, 1].values[0]
                    else:
                        value = 0 if feature == "Inventory" else np.nan  # Set missing Inventory to 0
                except Exception as e:
                    print(f"Error reading {feature} from {file}: {e}")
                    value = np.nan  # Default missing data to NaN

                # Ensure numeric conversion
                try:
                    data[feature] = float(value)
                except ValueError:
                    data[feature] = np.nan  # Handle non-numeric values

            # Compute derived metrics (handling missing values)
            try:
                data["Net Profit Margin %"] = (data["Net Income Continuous Operations"] / data["Total Revenue"]) * 100
                data["Return on Equity %"] = (data["Net Income Continuous Operations"] / data["Stockholders Equity"]) * 100
                data["Return on Assets %"] = (data["Net Income Continuous Operations"] / data["Total Assets"]) * 100
                data["Current Ratio"] = data["Current Assets"] / data["Current Liabilities"]
                data["Quick Ratio"] = (data["Current Assets"] - data["Inventory"]) / data["Current Liabilities"]
                data["Asset Turnover Ratio"] = data["Total Revenue"] / data["Total Assets"]

                # Handle division by zero in Interest Coverage Ratio
                data["Interest Coverage Ratio"] = np.where(
                    data["Interest Expense"] == 0, np.nan, data["EBIT"] / data["Interest Expense"]
                )

            except ZeroDivisionError:
                pass  # Already handled by numpy

            # Store final row
            data["Company Name"] = file.replace("_Financials.xlsx", "")  # Extract company name from filename
            final_data.append(data)

        except Exception as e:
            print(f"Error processing {file}: {e}")

# Convert to DataFrame
df_final = pd.DataFrame(final_data)

# # Save the cleaned dataset
# df_final.to_csv("processed_financials.csv", index=False)

# print("Processing complete. Data saved to 'processed_financials.csv'.")


In [8]:
df_final

,Net Income Continuous Operations,Total Revenue,Stockholders Equity,Total Assets,Current Assets,Current Liabilities,Inventory,Total Debt,Interest Expense,EBIT,Net Profit Margin %,Return on Equity %,Return on Assets %,Current Ratio,Quick Ratio,Asset Turnover Ratio,Interest Coverage Ratio,Company Name
0,9.350000e+09,1.886230e+11,6.935400e+10,1.675310e+11,5.279700e+10,4.664500e+10,4.598000e+09,5.332600e+10,3.525000e+09,1.715000e+10,4.956978,13.481558,5.581057,1.131890,1.033315,1.125899,4.865248226950355,Apollo_Hospitals
1,2.634626e+09,2.234409e+10,1.218354e+10,1.533504e+10,4.962676e+09,2.174184e+09,8.214200e+08,1.663448e+09,7.396900e+07,3.553466e+09,11.791153,21.624476,17.180433,2.282546,1.904740,1.457061,48.03993564871771,Bikaji_Foods_Limited
2,2.781810e+09,1.417252e+10,1.851342e+10,5.749547e+10,8.821130e+09,1.912796e+10,5.420400e+09,2.804836e+10,1.966040e+09,4.659570e+09,19.628196,15.025911,4.838312,0.461164,0.177788,0.246498,2.3700280767430977,Chalet_Hotels_Limited
3,8.356000e+09,9.179000e+10,3.626600e+10,6.107700e+10,2.602500e+10,1.531500e+10,0.000000e+00,7.260000e+09,1.205000e+09,1.165400e+10,9.103388,23.040865,13.681091,1.699314,1.699314,1.502857,9.671369294605809,Coforge
4,1.262090e+10,7.964750e+10,1.182321e+11,1.403804e+11,4.285970e+10,1.245700e+10,5.022000e+08,9.144800e+09,7.049000e+08,1.739030e+10,15.845946,10.674681,8.990500,3.440612,3.400297,0.567369,24.670591573272805,Container_Corpn
5,-2.491860e+09,8.141538e+10,9.144646e+10,1.145302e+11,5.950185e+10,1.346523e+10,1.642600e+08,1.169340e+10,8.775400e+08,-1.566940e+09,-3.060675,-2.724939,-2.175723,4.418926,4.406727,0.710864,-1.7856052145771133,Delhivery_Limited
6,2.127940e+09,1.053410e+10,3.420106e+10,3.926549e+10,3.136830e+10,4.103390e+09,0.000000e+00,2.551100e+08,2.574000e+07,2.472000e+09,20.200492,6.221854,5.419364,7.644484,7.644484,0.268279,96.03729603729603,Den_Networks_Limited
7,6.452191e+09,6.835087e+10,7.662897e+10,1.328878e+11,1.420589e+10,3.170169e+10,1.074164e+09,1.155017e+10,1.080475e+09,9.660138e+09,9.439809,8.420042,4.855367,0.448111,0.414228,0.514350,8.940639996297925,Fortis_Hospitals
8,4.117000e+09,2.633080e+10,9.712700e+09,1.830750e+10,1.186610e+10,7.590400e+09,3.958400e+09,5.000000e+05,8.630000e+07,5.708800e+09,15.635681,42.387802,22.488051,1.563304,1.041803,1.438252,66.15063731170336,Gillette_India_Limited
9,8.977293e+10,NaN,6.320756e+11,5.150940e+12,NaN,NaN,0.000000e+00,4.761141e+11,2.513229e+11,NaN,NaN,14.202878,1.742846,NaN,NaN,NaN,nan,IndusindBank


In [9]:
df_final.to_excel("processed_financials.xlsx", index=False)

In [2]:
!pip install pandas

^C


  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
     -------------------------------------- 509.2/509.2 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 347.8/347.8 kB 5.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
